
# Generate Readme with up to date list of xRunBooks

## Input

### Import librairies

In [1]:
import os
import requests
import urllib.parse
import copy
import markdown
import nbformat
from nbconvert import MarkdownExporter
from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)
try:
    from git import Repo
except:
    !pip install GitPython
    from git import Repo

### Variables

In [2]:
# README variables
readme_template = "README_template.md"
readme = "README.md"
replace_var = "[[DYNAMIC_LIST]]"

# welcome variables
#this is a TODO
#welcome_template = "Welcome_template.ipynb"
#welcome = "Welcome.ipynb"
#replace_var_quote = f'"[[DYNAMIC_LIST]]",\n'



# Others
current_file = '.'
notebook_ext = '.ipynb'
github_url = 'https://github.com/unskript/Awesome-CloudOps-Automation/tree/master'
local_OSS_url = 'http://127.0.0.1:8888/lab/tree'
#fix these!
github_download_url = 'https://raw.githubusercontent.com/unskript/Awesome-CloudOps-Automation/master/'
unSkript_logo ='https://unskript.com/assets/favicon.png'


### Get files list

In [3]:
repo = Repo('.')
branch = repo.active_branch
list_of_dir = f"https://api.github.com/repos/unskript/Awesome-CloudOps-Automation/git/trees/{branch.name}?recursive=1"
r_gh = requests.get(list_of_dir).json().get("tree")
notebooks = []
actions=[]
for file in r_gh:
    if ".github" not in file.get("path") and ".gitignore" not in file.get("path") and "templates" not in file.get("path") and "/" in file.get("path")and "__init__" not in file.get("path"):
        if file.get("path").endswith(".ipynb") or file.get("path").endswith(".py"):
            temp = file.get("path").split("/")
            if temp == -1:
                data = {
                    "root": None,
                    "subdir": file.get("path")
                }
                notebooks.append(data)
                
            else:
                last_folder = ""
                file_name = temp[-1]
                temp.pop()
                for folder in temp:
                    last_folder += "/" + folder
                ##we now have a path.. but only really need the root folder
                ## different ways to generate fior action vs runbook
                slashcount  = last_folder.count("/")
                if slashcount == 1:
                    ##root folder for notebooks
                    root = last_folder[1:]
                else:
                    #this is an action folder
                    #find frist slash
                    firstslash = last_folder.find("/",1)
                    root = last_folder[1:firstslash]
                
                data = {
                    "root": root,
                    "subdir": file_name
                }
                if(file.get("path").endswith(".ipynb")):
                    notebooks.append(data)
                else:
                    actions.append(data)
print(notebooks)
print(actions)

[{'root': 'AWS', 'subdir': 'AWS_Access_Key_Rotation.ipynb'}, {'root': 'AWS', 'subdir': 'Add_new_IAM_user.ipynb'}, {'root': 'AWS', 'subdir': 'Configure_url_endpoint_on_a_cloudwatch_alarm.ipynb'}, {'root': 'AWS', 'subdir': 'Copy_ami_to_all_given_AWS_regions.ipynb'}, {'root': 'AWS', 'subdir': 'Delete_Unattached_EBS_Volume.ipynb'}, {'root': 'AWS', 'subdir': 'Detach_Instance_from_ASG_and_Load_Balancer.ipynb'}, {'root': 'AWS', 'subdir': 'Detect_ECS_failed_deployment.ipynb'}, {'root': 'AWS', 'subdir': 'EC2_Disk_Cleanup.ipynb'}, {'root': 'AWS', 'subdir': 'Enforce_HTTP_Redirection_across_AWS_ALB.ipynb'}, {'root': 'AWS', 'subdir': 'Enforce_Mandatory_Tags_Across_All_AWS_Resources.ipynb'}, {'root': 'AWS', 'subdir': 'Find_EC2_Instances_Scheduled_to_retire.ipynb'}, {'root': 'AWS', 'subdir': 'Get_Aws_Elb_Unhealthy_Instances.ipynb'}, {'root': 'AWS', 'subdir': 'Lowering_AWS_CloudTrail_Costs_by_Removing_Redundant_Trails.ipynb'}, {'root': 'AWS', 'subdir': 'Monitor_AWS_DynamoDB_provision_capacity.ipynb'},

## Model

### Reformat functions

In [4]:
def reformat_file_name(file):
    file_nice = file.replace('_', ' ')
    file_nice = file_nice.replace('-', ' ')
    file_nice = file_nice.replace(notebook_ext, '')
    file_nice = file_nice.strip()
    if (file_nice != ""):
            file_nice = file_nice[0].capitalize() + file_nice[1:]
    return file_nice

### Get functions

In [5]:

#title = get_title(folder_nice, file,file_nice, github_runbook_url, local_url)
def get_title(folder_nice, file,file_nice, github_runbook_url, local_url):
    return f'| {folder_nice} | [{file_nice}]({github_runbook_url}) | [Open in browser]({local_url})'

def get_list_item(folder_nice, file,file_nice, github_runbook_url, local_url):
    return f'* [{file_nice}]({github_runbook_url})'    

def create_directory_readme(folder_nice, runbook_list):
    # Open README template
    #template = open(directory_readme_template).read()
    
    #directory readme name
    directory_readme = f"{folder_nice}/README.md"
    print("directory_readme", directory_readme)
    template_header = f"<img align=\"center\" src=\"https://unskript.com/assets/favicon.png\" width=\"100\" height=\"100\" style=\"padding: 5px\">"    
    template = f"{template_header}\n\n # {folder_nice} xRunBooks\n\n{runbook_list}"
    print(template)
    
    # Save README
    f  = open(directory_readme, "w+")
    f.write(template)
    f.close()
    
def append_directory_readme(folder_nice, action_list):
    
    ##actions are written in a second loop - so append the readme - if it has the term xrunbook in it.
    #if there is no "xRunBook" - then this direcory only has actions and should overwrite the entire file
    
    # Open README (created with runbook list)
    #directory readme name
    directory_readme = f"{folder_nice}/README.md"
    template = open(directory_readme).read()
    print("directory_readme", directory_readme)
    
    if "xRunBook" in template:
        #we append the file
        ##now let's append in the Action list
        template = template + f"\n\n # {folder_nice} Actions \n\n{action_list}"
    else:
        #this file has no runbook data, so overwrite the file
        template =  f"\n\n # {folder_nice} Actions \n\n{action_list}"
    print(template)
    
    # Save README
    f  = open(directory_readme, "w+")
    f.write(template)
    f.close()

### Convert filepath in Markdown text

In [6]:
def get_file_md(folder_nice, folder_url, files):
    good_format = 0
    final_title = ""
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
       # md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
            if file.endswith(notebook_ext):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                
                #create the localhost url by combining the root with the file name
                local_url= f"{local_OSS_url}/{file_url}"
                github_runbook_url = f"{github_url}/{folder_name}/{file_url}"
                
                title = f"{get_title(folder_nice, file, file_nice, github_runbook_url, local_url)}\n"
                
                md += title

    #print("md", md)
    return md

In [7]:
def get_file_md_quote(folder_nice, folder_url, files):
    good_format = 0
    final_title = ""
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
       # md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
            if file.endswith(notebook_ext):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                
                #create the localhost url by combining the root with the file name
                local_url= f"{local_OSS_url}/{file_url}"
                github_runbook_url = f"{github_url}/{folder_name}/{file_url}"        
                title = f'"{get_title(folder_nice, file, file_nice, github_runbook_url, local_url)}",\n'      
                md += title

   # print("md", md)
    return md

## get list of xRunbooks for subfolder Readme

In [8]:
def get_file_list(folder_nice, folder_url, files):
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
        for file in files:
            if file.endswith(notebook_ext):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                
                #create the localhost url by combining the root with the file name
                local_url= f"{local_OSS_url}/{file_url}"
                github_runbook_url = f"{github_url}/{folder_name}/{file_url}"
                
                list_item = f"* [{file_nice}]({github_url})\n"
                md += list_item

   # print("md", md)
   # print(json_templates)
    return md

## get list of Actions for subfolder Readme

In [9]:
def get_action_list(folder_nice, files):
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
        for file in files:
            if file.endswith(".py"):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                file_nice = reformat_file_name(file)
                list_item = f"* {file_nice}\n"
                md += list_item

   # print("md", md)
   # print(json_templates)
    return md

### Generate markdown for each notebooks

In [10]:
generated_list = ""
generated_list_quote = ""
json_templates = []
list_of_tools = []
index_max = len(notebooks)
index = 0

while index <= (index_max) - 1:
    folder_nice = notebooks[index].get("root")
    if folder_nice not in list_of_tools and folder_nice != "":
        #md_round is creating the table for the home readme
        md_round = ""
        md_round_quote=""
        #md_folder_readme is creating the simpler table for the subdirectory readme
        md_folder_readme = ''
        files = []
        list_of_tools.append(folder_nice)
        folder_url = urllib.parse.quote(folder_nice)
        print(folder_nice)
        while True:
            if notebooks[index].get("root") != folder_nice:
                break
            print(notebooks[index].get("subdir"))
            files.append(notebooks[index].get("subdir"))
            index += 1
            if index == index_max:
                break
            
        #create the table for the home readme
        md_round += get_file_md(folder_nice, folder_url, files )
        md_round_quote += get_file_md_quote(folder_nice, folder_url, files)
            
        #create a simple list for the subdirectory readme
        md_folder_readme += get_file_list(folder_nice, folder_url, files)
        #with this list - create the readme for the directory
        create_directory_readme(folder_nice, md_folder_readme)
    generated_list += md_round
  #  print("generated_list",generated_list)
    generated_list_quote += md_round_quote

AWS
AWS_Access_Key_Rotation.ipynb
Add_new_IAM_user.ipynb
Configure_url_endpoint_on_a_cloudwatch_alarm.ipynb
Copy_ami_to_all_given_AWS_regions.ipynb
Delete_Unattached_EBS_Volume.ipynb
Detach_Instance_from_ASG_and_Load_Balancer.ipynb
Detect_ECS_failed_deployment.ipynb
EC2_Disk_Cleanup.ipynb
Enforce_HTTP_Redirection_across_AWS_ALB.ipynb
Enforce_Mandatory_Tags_Across_All_AWS_Resources.ipynb
Find_EC2_Instances_Scheduled_to_retire.ipynb
Get_Aws_Elb_Unhealthy_Instances.ipynb
Lowering_AWS_CloudTrail_Costs_by_Removing_Redundant_Trails.ipynb
Monitor_AWS_DynamoDB_provision_capacity.ipynb
Notify_about_unused_keypairs.ipynb
Publicly_Accessible_Amazon_RDS_Instances.ipynb
Remediate_unencrypted_S3_buckets.ipynb
Renew_SSL_Certificate.ipynb
Resize_EBS_Volume.ipynb
Resize_List_Of_Pvcs.ipynb
Resize_PVC.ipynb
Restart_AWS_EC2_Instances.ipynb
Restart_Aws_Instance_given_Tag.ipynb
Restart_Unhealthy_Services_Target_Group.ipynb
Restrict_S3_Buckets_with_READ_WRITE_Permissions.ipynb
Run_EC2_from_AMI.ipynb
Secure_P

### Generate markdown for each Action

In [18]:
generated_list_actions = ""
generated_list_quote_actions = ""
json_templates = []
list_of_tools = []
index_max = len(actions)
index = 0

while index <= (index_max) - 1:
    folder_nice = actions[index].get("root")
    if folder_nice not in list_of_tools and folder_nice != "":
        #md_round is creating the table for the home readme
        md_round = ""
        md_round_quote=""
        #md_folder_readme is creating the simpler table for the subdirectory readme
        md_folder_readme = ''
        files = []
        list_of_tools.append(folder_nice)
        folder_url = urllib.parse.quote(folder_nice)
       # print(folder_nice)
        while True:
            if actions[index].get("root") != folder_nice:
                break
      #      print(actions[index].get("subdir"))
            files.append(actions[index].get("subdir"))
            index += 1
            if index == index_max:
                break
            
        #print("stuff", folder_nice, files)
        md_folder_readme += get_action_list(folder_nice, files)
        print("md_folder_readme", md_folder_readme)
        #with this list - create the readme for the directory
        append_directory_readme(folder_nice, md_folder_readme)
    generated_list_actions += md_round
  #  print("generated_list",generated_list)
    generated_list_quote_actions += md_round_quote

md_folder_readme * Aws apply default encryption for s3 bucket.py
* Aws attach ebs to instances.py
* Aws attach new policy to user.py
* Aws attach tags to resources.py
* Aws change acl permissions of bucket.py
* Aws check ssl certificate expiry.py
* Aws cloudwatch attach webhook notification to alarm.py
* Aws create access key.py
* Aws create bucket.py
* Aws create iam user.py
* Aws create login profile.py
* Aws delete access key.py
* Aws delete bucket.py
* Aws delete s3 bucket encryption.py
* Aws delete volume by id.py
* Aws deregister instances.py
* Aws detach ebs to instances.py
* Aws detach instances from autoscaling group.py
* Aws ebs modify volume.py
* Aws ecs describe task definition.py
* Aws ecs detect failed deployment.py
* Aws ecs service restart.py
* Aws ecs update service.py
* Aws eks copy pod logs to bucket.py
* Aws eks delete pod.py
* Aws eks get all dead pods.py
* Aws eks get all namespaces.py
* Aws eks get all pods.py
* Aws eks get deployments name.py
* Aws eks get node 

## Output

### Preview the generated list

### Generate readme for github repository

In [14]:
# Open README template
template = open(readme_template).read()

# Replace var to get list of templates in markdown format
template = template.replace(replace_var, generated_list)

# Save README
f  = open(readme, "w+")
f.write(template)
f.close()